In [1]:
import pandas as pd 
import numpy as np

In [2]:
raw_sessiondata = pd.read_csv("20210113-133853.csv", skiprows=6, delimiter = ';')

In [3]:
#Add a column for 'timestamps'
import time
import datetime
import ciso8601

timestamp = []
for i in range(len(raw_sessiondata.index)):
    t = raw_sessiondata['PC-TIME'].iloc[i]
    ts= ciso8601.parse_datetime(t)
    if i == 0:
        t0= time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(t0-t0,3))
    else:
        tnow = time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(tnow-t0,3))

raw_sessiondata['timestamps'] = timestamp

In [4]:
df_group_info = raw_sessiondata.groupby('+INFO')
df_group_type = raw_sessiondata.groupby('TYPE')

## Stimulus (Audio) Time

In [5]:
# Find Audio timestamps ('BNC1High'), process so no duplicates
df_bnc1high = df_group_info.get_group('BNC1High')
a = df_bnc1high['timestamps'].diff().fillna(df_bnc1high['timestamps']) > 10
timestamps_bnc1high = df_bnc1high.loc[a].timestamps.to_numpy()

In [8]:
timestamps_bnc1high

array([  21.372,   62.148,   82.933,  113.702,  134.487,  165.471,
        196.28 ,  227.179,  248.061,  278.855,  319.656,  360.397,
        391.093,  411.841,  452.546,  483.274,  513.971,  534.681,
        565.468,  586.184,  616.935,  657.647,  688.392,  729.247,
        750.082,  770.789,  811.516,  852.292,  873.024,  913.73 ,
        954.466,  995.174, 1015.896, 1056.647, 1077.386, 1118.07 ,
       1138.835, 1159.552, 1200.25 , 1221.003, 1251.744, 1282.486,
       1303.243, 1323.968, 1354.675, 1395.437, 1426.174, 1446.901,
       1487.651, 1518.385, 1559.126, 1589.91 , 1610.663, 1631.424,
       1672.163, 1712.863, 1743.588, 1764.35 , 1785.127, 1815.884,
       1846.67 , 1887.433, 1928.166, 1958.91 , 1979.65 , 2010.377,
       2051.096, 2091.835, 2112.751, 2143.564, 2174.396, 2195.168,
       2225.922, 2266.626, 2307.507])

## Lick Response Time

In [9]:
# Get lick onsets 
df_port1out = df_group_info.get_group('Port1Out')
timestamps_lickon = df_port1out.timestamps.to_numpy()

#### In response to 1st stimulus -> 8 licks

In [22]:
timestamps_lickon[6:15]

array([21.611, 21.751, 21.936, 22.075, 22.253, 22.385, 22.555, 25.177,
       34.642])

#### In between 1st - 2nd stimulus -> 7 licks

In [40]:
timestamps_lickon[14:22]

array([34.642, 34.785, 35.006, 35.209, 47.2  , 52.594, 52.849, 62.371])

#### In response to 2nd stimulus -> 23 licks

In [33]:
timestamps_lickon[21:45]

array([62.371, 62.529, 62.71 , 62.855, 63.009, 63.177, 63.333, 63.494,
       63.634, 63.797, 63.974, 64.134, 64.306, 64.477, 64.648, 64.84 ,
       65.004, 65.184, 65.347, 65.523, 65.696, 65.863, 66.416, 83.178])

#### In between 2nd - 3rd stimulus -> 0 licks!

## Finding: In response to the audio stimulus:
1. sometimes they will lick more
2. sometimes they will lick in between sessions, and sometimes they don't

## How do their licking behavior (frequency, number) changes as more audio is being presented?

## How does the number of in-between stimulus response lick changes?

In [8]:
# intialise data of lists. 
data = {'lick_time':timestamps_lickon} 
# Create DataFrame 
df = pd.DataFrame(data)

In [9]:
df

lick_time
0         7.912
1         8.191
2         8.541
3         8.723
4        16.754
...         ...
2131   2307.753
2132   2307.879
2133   2308.027
2134   2308.196
2135   2308.301

[2136 rows x 1 columns]